# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,53.71,67,71,12.93,AU,1698606071
1,1,blackmans bay,-43.0167,147.3167,55.53,62,100,8.16,AU,1698606200
2,2,i-n-salah,27.1935,2.4607,88.02,15,100,5.12,DZ,1698606201
3,3,port mathurin,-19.6833,63.4167,76.23,79,12,15.03,MU,1698606068
4,4,port-aux-francais,-49.3500,70.2167,35.44,89,100,25.05,TF,1698606201


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot_cities = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles="OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City" 


)

# Display the map
map_plot_cities

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = city_data_df[city_data_df['Max Temp'] >= 60]

# Drop any rows with null values
min_temp.dropna()

# Display sample data
min_temp.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,i-n-salah,27.1935,2.4607,88.02,15,100,5.12,DZ,1698606201
3,3,port mathurin,-19.6833,63.4167,76.23,79,12,15.03,MU,1698606068
6,6,pitimbu,-7.4706,-34.8086,84.79,69,100,17.65,BR,1698606202
8,8,laguna,38.4210,-121.4238,70.09,26,0,8.05,US,1698606203
9,9,shakawe,-18.3667,21.8500,82.06,38,94,10.65,BW,1698606204


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = min_temp[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df.head()

C:\Users\jessi\AppData\Local\Temp\ipykernel_4168\3291307122.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=""


,City,Country,Lat,Lng,Humidity,Hotel Name
2,i-n-salah,DZ,27.1935,2.4607,15,
3,port mathurin,MU,-19.6833,63.4167,79,
6,pitimbu,BR,-7.4706,-34.8086,69,
8,laguna,US,38.4210,-121.4238,26,
9,shakawe,BW,-18.3667,21.8500,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
base_url = "https://api.geoapify.com/v2/places"

In [46]:
# Set parameters to search for a hotel
radius = 1000
categories = "accommodation.hotel"
conditions = "internet_access"

filters = f"circle:{hotel_df['Lng']},{hotel_df['Lat']},{radius}"
bias = f"proximity:{hotel_df['Lng']},{hotel_df['Lat']}"
limit = 50

params = {"categories":categories,
    "conditions":conditions,
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
i-n-salah - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
pitimbu - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
shakawe - nearest hotel: No hotel found
campo grande - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
port douglas - nearest hotel: No hotel found
tadine - nearest hotel: No hotel found
paragominas - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
queimadas - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
yaren - nearest hotel: No hotel found
las vegas - nearest hotel: No hotel found
perry - nearest hotel: No hotel found
saint-pierre - nearest hotel: Le Terre Sainte
lompoc - nearest hotel: No hotel found
voi - nearest hotel: No hotel found
castillos - nearest hotel: No hotel found
mazara del vallo - nearest hotel: No hotel found
amlapura -

rio grande - nearest hotel: Hotel Swan Expres
colonia menno - nearest hotel: No hotel found
keningau - nearest hotel: OYO 89577 Kristal Hotel
ad dawadimi - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
bikenibeu village - nearest hotel: No hotel found
saint-joseph - nearest hotel: No hotel found
buka - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
aioun - nearest hotel: No hotel found
tyuratam - nearest hotel: No hotel found
garoowe - nearest hotel: xaramayn hotel
tegal - nearest hotel: No hotel found
mata-utu - nearest hotel: Gîte Oceania
sirvan - nearest hotel: No hotel found
manokwari - nearest hotel: No hotel found
pozanti - nearest hotel: No hotel found
howard springs - nearest hotel: No hotel found
dien bien phu - nearest hotel: No hotel found
los frentones - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
siniscola - nearest hotel: No hotel found
malanje - nearest hotel: Hotel PORTVGALIA
monghyr - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
2,i-n-salah,DZ,27.1935,2.4607,15,No hotel found
3,port mathurin,MU,-19.6833,63.4167,79,No hotel found
6,pitimbu,BR,-7.4706,-34.8086,69,No hotel found
8,laguna,US,38.4210,-121.4238,26,No hotel found
9,shakawe,BW,-18.3667,21.8500,38,No hotel found
...,...,...,...,...,...,...
588,dayrut,EG,27.5560,30.8076,41,No hotel found
589,jiangyou,CN,31.7667,104.7167,87,No hotel found
590,maba,CN,24.6841,113.5984,88,No hotel found
591,tianpeng,CN,30.9866,103.9393,81,No hotel found


In [55]:
hotels_only = hotel_df[hotel_df['Hotel Name'] !='No hotel found']

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles="OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Hotel",
    scale = 15,
    color = "Hotel Name",
    hover_cols=["Hotel Name","Country"]

)

# Display the map
map_plot_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)

In [57]:
%%capture --no-display
#just for fun, getting only the hotels that meet requirements on the map 
# Configure the map plot
map_plot_hotels_only = hotels_only.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles="OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Hotel",
    scale = 15,
    color = "Hotel Name",
    hover_cols=["Hotel Name","Country"]

)

# Display the map
map_plot_hotels_only

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)